In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np


class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        return text, label

# 70で作成した行列をロード
X_train = np.load("./matrix/x_train.npy")
Y_train = np.load("./matrix/y_train.npy")

X_train_tensor = torch.from_numpy(X_train)
Y_train_tensor =torch.from_numpy(Y_train)
datasets = TextDataset(X_train, Y_train)
train_dataloader = DataLoader(datasets, shuffle=True, batch_size=64)

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(300, 4),
            nn.Softmax()
        )

    def forward(self, x):
        logits = self.layer(x)
        return logits

model = Model()
learning_rate = 1e-3
batch_size = 64
epochs = 100
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [2]:
from torch.utils.tensorboard import SummaryWriter

# SummaryWriterのインスタンスを作成
writer = SummaryWriter(log_dir="runs/75")

# train
for t in range(epochs):
    size = len(datasets)
    epoch_loss = 0
    epoch_correct = 0
    total_samples = 0

    for batch, (X, y) in enumerate(train_dataloader):
        # 予測と損失の計算
        X = X.float()
        pred = model(X)
        loss = loss_fn(pred, y.to(torch.float64))

        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 損失の累計
        epoch_loss += loss.item()

        # 正答数の累計
        predicted = pred.argmax(dim=1)
        correct = (predicted == y.argmax(dim=1)).sum().item()
        epoch_correct += correct
        total_samples += len(y)

    # エポックごとの平均損失と正答率を計算
    avg_loss = epoch_loss / len(train_dataloader)
    avg_accuracy = epoch_correct / total_samples

    #TensorBoardに記録
    writer.add_scalar('Loss/train', avg_loss, t)
    writer.add_scalar('Accuracy/train', avg_accuracy, t)


/Users/tajirimanato/.pyenv/versions/3.12.2/lib/python3.12/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [3]:
# test
for t in range(epochs):
    size = len(datasets)
    epoch_loss = 0
    epoch_correct = 0
    total_samples = 0

    for batch, (X, y) in enumerate(train_dataloader):
        # 予測と損失の計算
        X = X.float()
        pred = model(X)
        loss = loss_fn(pred, y.to(torch.float64))

        # 損失の累計
        epoch_loss += loss.item()

        # 正答数の累計
        predicted = pred.argmax(dim=1)
        correct = (predicted == y.argmax(dim=1)).sum().item()
        epoch_correct += correct
        total_samples += len(y)

    # エポックごとの平均損失と正答率を計算
    avg_loss = epoch_loss / len(train_dataloader)
    avg_accuracy = epoch_correct / total_samples

    #TensorBoardに記録
    writer.add_scalar('Accuracy/test', avg_accuracy, t)